[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/piebro/physarum-latent-walk/blob/master/example.ipynb)

# Setup

In [1]:
# install physarum latent walk lib
!git clone https://github.com/piebro/physarum-latent-walk.git physarum_latent_walk
!pip install -r physarum_latent_walk/requirements.txt -q

Cloning into 'physarum_latent_walk'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 56 (delta 9), reused 26 (delta 6), pack-reused 25
Unpacking objects: 100% (56/56), done.
     |████████████████████████████████| 40.6MB 1.4MB/s 


In [2]:
# download data from https://www.kaggle.com/pietbroemmel/physarum-latent-walk
# like this: https://www.kaggle.com/general/74235 with 
# or from drive like this:
!pip install gdown
!gdown --id "1snpDTbk2ZW6ZZ8sDU_xjTLzp1cwCM7qw"

Downloading...
From: https://drive.google.com/uc?id=1snpDTbk2ZW6ZZ8sDU_xjTLzp1cwCM7qw
To: /content/physarum-dataset.zip
3.97GB [00:35, 113MB/s] 


In [3]:
!unzip physarum-dataset.zip

Archive:  physarum-dataset.zip
 extracting: physarum-dataset/cells.zip  
 extracting: physarum-dataset/colorful_cells.zip  
 extracting: physarum-dataset/duckwheat.zip  
 extracting: physarum-dataset/dunes.zip  
 extracting: physarum-dataset/fire.zip  
 extracting: physarum-dataset/giraffe.zip  
 extracting: physarum-dataset/grid.zip  
 extracting: physarum-dataset/labyrinth.zip  
 extracting: physarum-dataset/layer.zip  
 extracting: physarum-dataset/plant_cells.zip  
 extracting: physarum-dataset/random.zip  
 extracting: physarum-dataset/red_flow.zip  
 extracting: physarum-dataset/runes.zip  
 extracting: physarum-dataset/space1.zip  
 extracting: physarum-dataset/space2.zip  
 extracting: physarum-dataset/sponge.zip  
 extracting: physarum-dataset/thunder.zip  
 extracting: physarum-dataset/universe.zip  
 extracting: physarum-dataset/vines.zip  
 extracting: physarum-dataset/wetlands.zip  


In [4]:
import sys
import os
sys.path.append(os.path.abspath("physarum_latent_walk"))

import tools
import embedding_transformation as et
import model

exp_dir = "/content/experiments/"
dataset_dir = "/content/physarum-dataset"
if not os.path.isdir(exp_dir): os.mkdir(exp_dir)

# Train

In [5]:
# in general my experiances with the model and the dataset are:
# - for a fine structure use less depth (4 or 5) and for a coarse strucure with less details use more depth (6 or 7)
# - a latent space of 4096 worked well for me. This means:
#     depth:4 => last_filter:1
#     depth:5 => last_filter:4
#     depth:6 => last_filter:16
#     depth:7 => last_filter:64
# - batch_size of 1 works the best most of the time
# - the "zoom_translate" augmentation works the best
# - longer training doens't always mean better images. The images get more details but the interpolation get worse sometimes.
# - batch_norm doesn't improve training

In [6]:
args = {
  "img_dir": "runes",
  "depth": 6,
  "last_filter_size": 16,
  "filter_size": 64,
  "batch_norm": False,
  "epochs": 5,
  "batch_size": 1,
  "steps_per_epoch": 400,
  "img_size": 1024,
  "augmentation": "zoom_translate",
}
run_id = model.run(exp_dir, dataset_dir, args)

run_dir: /content/experiments/1a94750dfa157cde44af28be0b084743
Epoch 1/5
400/400 [==============================] - 99s 238ms/step - binary_crossentropy: 522658.4688 - val_binary_crossentropy: 497493.0625
Epoch 2/5
400/400 [==============================] - 94s 235ms/step - binary_crossentropy: 496635.0000 - val_binary_crossentropy: 494786.3125
Epoch 3/5
400/400 [==============================] - 94s 235ms/step - binary_crossentropy: 495534.8438 - val_binary_crossentropy: 494200.6562
Epoch 4/5
400/400 [==============================] - 94s 235ms/step - binary_crossentropy: 495367.0312 - val_binary_crossentropy: 493985.9375
Epoch 5/5
400/400 [==============================] - 94s 235ms/step - binary_crossentropy: 494786.2500 - val_binary_crossentropy: 494248.5000


In [ ]:
# example of embedding transformations
framerate = 15
num_of_images = 4
et_linear = [
  et.init_embedding_count(num_of_images),
  et.reorder_with_tsp(),
  et.interpolate_smooth_equi_dist(frame_count=8*framerate, smoothness=0.25, num_of_iterations=2),        
]
et_cluster_linear = [
  et.init_embedding_count(60),
  et.cluster(num_of_images),
  et.reorder_with_tsp(),
  et.interpolate_smooth_equi_dist(frame_count=8*framerate, smoothness=0.25, num_of_iterations=2),
]
et_noise = [
  et.init_embedding_count(num_of_images),
  et.reorder_with_tsp(),
  et.loopable_noise(num_of_images*framerate, local_factor=2, noise_speed=0.05, noise_inpact=2)
]

# create video
video_path = model.save_video(exp_dir, dataset_dir, run_id, et_noise, framerate=framerate, video_name="et_noise", epoch=None)

video path:  /content/experiments/1a94750dfa157cde44af28be0b084743/et_noise_seed121397_epoch005.mp4


In [12]:
tools.show_video_html(video_path)

In [11]:
# continue training
run_id = model.continue_training(exp_dir, dataset_dir, run_id, 5)

continue training on: /content/experiments/1a94750dfa157cde44af28be0b084743
Epoch 6/10
400/400 [==============================] - 94s 236ms/step - binary_crossentropy: 494854.9688 - val_binary_crossentropy: 493752.8438
Epoch 7/10
400/400 [==============================] - 94s 236ms/step - binary_crossentropy: 494590.5938 - val_binary_crossentropy: 493556.6562
Epoch 8/10
400/400 [==============================] - 94s 236ms/step - binary_crossentropy: 494587.1562 - val_binary_crossentropy: 493942.5938
Epoch 9/10
400/400 [==============================] - 94s 235ms/step - binary_crossentropy: 494378.5312 - val_binary_crossentropy: 493822.7500
Epoch 10/10
400/400 [==============================] - 94s 235ms/step - binary_crossentropy: 494308.3125 - val_binary_crossentropy: 493377.8438


# Util

In [ ]:
from keras.preprocessing import image

datasets_str = """
cells
colorful_cells
duckwheat
dunes
fire
giraffe
grid
labyrinth
layer
plant_cells
red_flow
runes
space1
space2
sponge
thunder
universe
vines
wetlands
"""

def get_datasets_example(datasets, dataset_dir):
  for dataset_name in datasets: model.unpack_dataset(dataset_dir, dataset_name)
  dataset_dirs = sorted(os.path.join(dataset_dir, dataset) for dataset in datasets)
  img_paths = [os.path.join(dd, os.listdir(dd)[0]) for dd in dataset_dirs]
  imgs = [image.load_img(img_path, color_mode="rgb") for img_path in img_paths]
  return imgs

datasets = datasets_str.split("\n")[1:-1]
fig = tools.get_figure_of_images(get_datasets_example(datasets, dataset_dir), titles=datasets, height=None, width=4, size=5)

In [ ]:
# plot loss
df = model.get_experiment_dataframes(exp_dir)
run_df = df[df["exp_id"]==run_id]
print("dataset: ", run_df['img_dir'].item())
model.plot_loss(df, run_id)